In [ ]:
import tensorflow_datasets as tfds

In [ ]:
import tensorflow as tf
data_ds = tf.data.Dataset.list_files("../data/ILSVRC-2012/val/*")  # already gets it i the correct order for zip
print(type(data_ds))
print(len(data_ds))



In [ ]:
with open("../data/ILSVRC-2012/ILSVRC2012_devkit_t12/data/ILSVRC2012_validation_ground_truth.txt") as f:
    labels = list(f) 
print(len(labels))
label_ds = tf.data.Dataset.from_tensor_slices(labels)
print(len(label_ds))
print(type(label_ds))

In [ ]:
ds = tf.data.Dataset.zip((data_ds, label_ds))

In [ ]:
# TODO test order to be sure things match up right

In [ ]:
def convert_image(file_name, *args):
    img = read
    return file_name, *args

In [ ]:
print("pls")

In [ ]:
import tensorflow as tf
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, rotation_range=20, horizontal_flip=True)
ds = img_gen.flow_from_directory('../data/ILSVRC-2012', )



In [ ]:
img, label = next(ds)
label

In [ ]:
tf.__version__

In [ ]:
import scipy.io
meta = scipy.io.loadmat('../data/external/ILSVRC-2012/ILSVRC2012_devkit_t12/data/meta.mat', squeeze_me=True)['synsets']

In [ ]:
meta

In [ ]:
meta[0][2]

In [ ]:
meta[1][0]

In [ ]:
data = meta[0,  0][1]
data

In [ ]:

len(meta)

In [ ]:
tags = [meta[idx][0] for idx in range(len(meta))]
#tags

In [ ]:
meta.__dict__

In [ ]:
len(meta)

In [ ]:
meta

In [ ]:
import tensorflow as tf
files = tf.data.Dataset.list_files('../data/external/ILSVRC-2012/train/*')

In [ ]:
print(type(files))
print(len(files))

In [ ]:

item = next(iter(files))
print(item)
file_name = tf.strings.split(item, sep='/')[-1]
print(file_name)
wnid, *_ = tf.strings.split(file_name, sep='_')
print(wnid)

In [ ]:
import tensorflow as tf
import pandas as pd

import pandas as pd
meta = pd.read_csv('../data/interim/meta.csv')
#wnid_to_id = tf.lookup.StaticHashTable({wnid: id for id, wnid in zip(meta.id.values, meta.wnid.values)}, default_value=-1)
ids = [id-1 for id in meta.id.values]
wnid_to_id = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        values=ids, keys=meta.wnid.values), 
    default_value=-1)

def get_id(wnid):
    #id = tf.py_function(lambda x: wnid_to_id[x], [wnid], tf.int32)
    #return id
    return wnid_to_id.lookup(wnid)

def get_wnid(path):
    file_name = tf.strings.split(path, sep='/')[-1]
    split_name = tf.strings.split(file_name, sep='_')
    wnid = split_name[0]
    return wnid

# TODO test setting shape
def img_decode(img_bytes):
    img = tf.image.decode_jpeg(img_bytes, channels=3)
    img = tf.image.convert_image_dtype(img, 'float32')
    img = tf.image.resize(img, (224, 224))
    # TODO check if shape is greater than a certain amount, if so random crop,
    # TODO else just resize
    # TODO make sure mutations don't go above/below 0/1
    return img


def loader(path):
    files_ds = tf.data.Dataset.list_files(path)
    data_ds = files_ds.map(tf.io.read_file, tf.data.AUTOTUNE).filter(tf.io.is_jpeg)
    img_ds = data_ds.map(img_decode, tf.data.AUTOTUNE)

    wnid_ds = files_ds.map(get_wnid, tf.data.AUTOTUNE)
    ids_ds = wnid_ds.map(get_id, tf.data.AUTOTUNE)
    print(len(wnid_ds))
    ds = tf.data.Dataset.zip((img_ds, ids_ds))

    return ds
ds = loader('../data/external/ILSVRC-2012/train/*')
print(type(ds))

In [ ]:
next(iter(ds))

In [ ]:
#model = tf.keras.applications.ResNet50()
model= tf.keras.applications.EfficientNetB0()

model.compile(
    metrics=['acc'],
    loss=tf.keras.losses.SparseCategoricalCrossentropy())
model.summary()

In [ ]:
model.fit(ds.batch(4).prefetch(-1), epochs=10)